## Project some statistical results on the freesurfer surfaces

In [2]:
# initiate some paths and files
rt=/Users/chris/Desktop/stats2surf

In [ ]:
fslmaths ${rt}/epi.nii.gz -mas ${rt}/epi_mask.nii.gz ${rt}/epi_brain.nii.gz
cp ${rt}/epi_brain.nii.gz ${rt}/epi_brain_rescale.nii.gz

In [ ]:
# create a 1mm (header) version of the epi
cp ${rt}/results/MeanAngle_th4_pv0.2.nii.gz ${rt}/results/MeanAngle_th4_pv0.2_rescale.nii.gz
cp ${rt}/results/tstat1.nii.gz ${rt}/results/tstat1_rescale.nii.gz
cp ${rt}/results/tstat2.nii.gz ${rt}/results/tstat2_rescale.nii.gz

3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen ${rt}/epi_brain_rescale.nii.gz
3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen ${rt}/results/MeanAngle_th4_pv0.2_rescale.nii.gz
3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen ${rt}/results/tstat1_rescale.nii.gz
3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen ${rt}/results/tstat2_rescale.nii.gz

In [3]:
# do a flirt registration on the orgiginal files (to the 1mm header files)
mkdir -p ${rt}/flirt
flirt -ref ${rt}/fsSurf/src/brain_org.nii.gz \
    -wmseg ${rt}/fsSurf/src/wm.nii.gz \
    -in ${rt}/epi_brain.nii.gz \
    -out ${rt}/flirt/epi2anat.nii.gz \
    -omat ${rt}/flirt/epi2anat.mat \
    -pedir -2      

In [ ]:
# do a flirt reistration on the 1mm voxel-header files
flirt -ref ${rt}/fsSurf/src/brain.nii.gz \
    -wmseg ${rt}/fsSurf/src/wm.nii.gz \
    -in ${rt}/epi_brain_rescale.nii.gz \
    -out ${rt}/flirt/epi2anat_rescale.nii.gz \
    -omat ${rt}/flirt/epi2anat_rescale.mat \
    -pedir -2  

In [ ]:
# tkregister
# convert flirt registration matrix to tkregister (and check registration)
tkregister2 --mov ${rt}/epi_brain_rescale.nii.gz \
    --targ ${rt}/fsSurf/src/brain.nii.gz \
    --fsl ${rt}/flirt/epi2anat_rescale.mat \
    --reg ${rt}/reg.fsl.dat \
    --s Danny

In [ ]:
mri_vol2surf --trgsubject Danny \
    --src ${rt}/results/tstat1_rescale.nii.gz \
    --out ${rt}/results/targ1.w --out_type paint \
    --srcreg ${rt}/reg.fsl.dat \
    --hemi lh
mri_vol2surf --trgsubject Danny \
    --src ${rt}/results/tstat2_rescale.nii.gz \
    --out ${rt}/results/targ2.w --out_type paint \
    --srcreg ${rt}/reg.fsl.dat \
    --hemi lh    
mri_vol2surf --trgsubject Danny \
    --src ${rt}/results/MeanAngle_th4_pv0.2_rescale.nii.gz \
    --out ${rt}/results/angle_map.w --out_type paint \
    --srcreg ${rt}/reg.fsl.dat \
    --hemi lh

In [ ]:
# vol2surf
tksurfer Danny lh graymid \
-patch full.patch.flat \
-overlay ${rt}/results/targ2.w \
-overlay-reg ${rt}/reg.fsl.dat 

In [ ]:
tkregister2 --sd ${targ_dir} --s ${structural_session} --targ ${targ_dir}/${targ_vol} --mov ${mov_dir}/${mov_vol} --reg ${out_reg_dir}/${out_reg} --regheader --movscale 2 ${tkregister2_opts}#--noedit
